In [1]:
from google.colab import drive
import sqlite3
import pandas as pd

#подгружаем файлы
drive.mount('/content/drive')
!ls /content/drive/MyDrive/Colab/GlowByte/

Mounted at /content/drive
3612_ТЗ_Команда_11_v1.2.docx  Python_SQL-скрипт.txt  SQL-скрипт.txt   промежточные
database.db		      Screenshot.jpg	     Untitled0.ipynb


In [3]:
# Путь к базе данных SQLite
dbfile = '/content/drive/MyDrive/Colab/GlowByte/database.db'

# Создаем соединение с базой данных
conn = sqlite3.connect(dbfile)

# Выполнение запроса и сохранение результатов в датафрейм
df_3612 = pd.read_sql_query("""
-- Определяем временную таблицу "SuspiciousOperations", чтобы собрать подозрительные операции
WITH SuspiciousOperations AS (
    SELECT
        fl.full_name,
        fl.okved_code,
        t.client_id,
        t.payment_purpose,
        t.operation_status,
        -- Суммируем количество операций, конвертируя суммы из иностранной валюты в рубли, если необходимо
        SUM(CASE
            WHEN t.currency = 'RUB' THEN t.operation_amount
            ELSE t.operation_amount * t.exchange_rate
        END) AS total_amount,
        -- Считаем количество операций
        COUNT(t.operation_id) AS operation_count,
        -- Получаем дату первой операции
        MIN(t.operation_date) AS first_operation_date,
        -- Получаем дату последней операции
        MAX(t.operation_date) AS last_operation_date,
        -- Соединяем все идентификаторы операций в одну строку, разделенную запятой
        GROUP_CONCAT(t.operation_id, ', ') AS operation_ids,
        -- Устанавливаем тип клиента как "ФЛ" (физическое лицо)
        'ФЛ' AS client_type
    -- Исходная таблица с данными о транзакциях
    FROM
        "transaction" t
    -- Соединяем таблицу клиентов с транзакциями по идентификатору клиента
    JOIN
        client_fl fl ON t.client_id = fl.client_id
    -- Фильтруем транзакции: выбираем только те, что были за последние 30 дней
    WHERE
        t.operation_date >= DATE('now', '-30 days')
        -- Учитываем только завершенные операции
        AND t.operation_status = 'ЗАВЕРШЕНА'
        -- Фильтруем по целям платежей, включая только определенные ключевые слова
        AND (
            t.payment_purpose LIKE '%лотерея%' OR
            t.payment_purpose LIKE '%тотализатор%' OR
            t.payment_purpose LIKE '%пари%' OR
            t.payment_purpose LIKE '%игр%' OR
            t.payment_purpose LIKE '%казино%' OR
            t.payment_purpose LIKE '%ставка%' OR
            t.payment_purpose LIKE '%азарт%' OR
            t.payment_purpose LIKE '%рубль%' OR
            t.payment_purpose LIKE '%выигрыш%' OR
            t.payment_purpose LIKE '%приз%' OR
            t.payment_purpose LIKE '%игровой%' OR
            t.payment_purpose LIKE '%фарт%' OR
            t.payment_purpose LIKE '%джекпот%'
        )
        -- Проверяем код ОКВЭД клиента; выбираем только некоторые коды или NULL
        AND (
            fl.okved_code IN (92.11, 92.12, 92.13, 92.21, 92.22, 92.23) OR fl.okved_code IS NULL
        )
    -- Группироваем результаты по клиенту
    GROUP BY
        t.client_id, client_type, fl.full_name
    -- Условия для выборки только результатов с как минимум двумя операциями и общей суммой операций 600000 или более
    HAVING
        operation_count >= 2
        AND total_amount >= 600000
)
-- Основной запрос для извлечения данных из временной таблицы SuspiciousOperations
SELECT
    -- Выводим уникальный идентификатор клиента
    so.client_id,
    -- Выводим строку со всеми идентификаторами операций, которые соответствуют критериям
    COALESCE(NULLIF(so.operation_ids, ''), 'Нет операций') AS operation_ids,
    -- Формируем сообщение о подозрительной операции с информацией о числе совершенных операций и общей сумме
    'Подозрительная операция по коду 3612: Клиент совершил ' ||
    COALESCE(so.operation_count, 0) ||
    ' транзакций на общую сумму ' ||
    COALESCE(so.total_amount, 0) ||
    ' RUB, превышающую допустимый лимит, в течение 30 дней.' AS Scenario_3612
FROM
    SuspiciousOperations so
-- Сортируем результаты по общей сумме операций в порядке убывания
ORDER BY
    so.total_amount DESC;
""", conn)

# Закрываем соединение с базой данных
conn.close()

# Печать первых строк для проверки
print(df_3612.shape)
df_3612.head()

(144, 3)


,client_id,operation_ids,Scenario_3612
0,49730_UCJ,"732803_LAF, 883638_GOB, 914911_ABP, 933567_QOG",Подозрительная операция по коду 3612: Клиент с...
1,77449_VQF,"153490_NPL, 827078_YKR, 629832_XMR, 191542_TTC",Подозрительная операция по коду 3612: Клиент с...
2,99241_LTG,"328134_HOV, 881328_YOV, 797722_GIM",Подозрительная операция по коду 3612: Клиент с...
3,17736_TIG,"278219_QGA, 199411_TMC, 893926_TIZ, 778436_EZR",Подозрительная операция по коду 3612: Клиент с...
4,68404_XBS,"982683_ABL, 243505_KBN, 280623_XAH",Подозрительная операция по коду 3612: Клиент с...
